In [ ]:
################################################################################################
# sentence-bert와 Faiss 라이브러리를 이용하여 검색 MRR(Mean Reciprocal Rank)와 BM25 측정하는 예시임
# => bm25 설치 : !pip install rank_bm25
#
# 1. 검색모델 로딩
# 2. json QuA파일 로딩 하여, 각 항목별 리스트를 출력한후, contexts df, questions df를 만듬.
# 3. 정답리스트에 대해 임베딩 벡터 생성 후 FAISS에 인덱싱함.
# 4. 쿼리를 list로 만들고, 검색 후 예측된 결과를 list로 만듬
# 5. 정답리스트와 예측리스트를 가지고 MRR을 구함
# 6. contexts df 를 BM25 인덱싱 하고 , questions df 를 쿼리로 입력하여 BM25 스코어 구함
#
# 쿼리가:단어일때
# => 문장 평균: 20%, 단어 평균: 10% 
# => BM25=Mecab 적용시: 85%, 적용안할때: 50%
#
#-------------------------------------------------------------------------------
# sklenarn 으로 cosine 확인 예제
# from sklearn.metrics.pairwise import cosine_similarity
# cosine_sim = cosine_similarity([embed_querys[0]], [embed_querys_1[0]]) # (1,768) 식에 2차원 배열입력되어야 함.
#-------------------------------------------------------------------------------
################################################################################################
import faiss
import numpy as np
import pandas as pd
import time
import json
import os
from tqdm.notebook import tqdm

from os import sys
sys.path.append('../../')
from myutils import GPU_info, seed_everything, mlogging

logger = mlogging(loggername="MRR-BM25", logfilename="../../../log/MRR-BM25")
device = GPU_info()
#device = 'cpu'  # cpu 테스트 할때

#------------------------------------------------------------------------------------
# 0. param 설정
#------------------------------------------------------------------------------------
seed = 111
query_num = 500            # 쿼리 최대 갯수: KorQuAD_v1.0_dev.json 최대값은 5533개임, 0이면 모든 5533개 쿼리함.
search_k = 5              # FAISS 검색시, 검색 계수(5=쿼리와 가장 근접한 5개 결과값을 반환함)
use_cross_encoder = False   # cross_encoder 사용할지 유.무 (true=사용함, false=사용안함)
use_bm25 = True           # BM25 출력 할지=True. 안할지=False
embed_avg_method = 1     # 0=문장 전체를 하나의 벡터로 생성(빠름), 1 = 문장 임베딩 구할때 여러문장 벡터 평균으로 구함(느림), 2=문장에 단어(명사) 추출후 단어 벡터들의 평균 구함(더느림)
faiss_index_method = 0   # Faiss 인덱스 생성 방식 => 0=코사인유사도(Cosine Similarity) 방식(IndexFlatIP 사용), 1= 유클리드거리(Euclidean Distance) 방식(IndexFlatL2 사용)
#------------------------------------------------------------------------------------

# param 인자 범위 체크
if faiss_index_method > 1 or faiss_index_method < 0:
    raise ValueError(f"faiss_index_method = {faiss_index_method} is not bad!!")

seed_everything(seed)

In [ ]:
#-------------------------------------------------------------------------------------
# 1.sentence bert 모델 로딩
#-------------------------------------------------------------------------------------
from myutils import bi_encoder
from sentence_transformers.cross_encoder import CrossEncoder
bi_encoder_path = "bongsoo/kpf-sbert-v1.1"#"bongsoo/kpf-sbert-v1.1" # kpf-sbert-v1.1 # klue-sbert-v1 # albert-small-kor-sbert-v1.1
pooling_mode = 'mean' # bert면=mean, albert면 = cls
out_dimension = 0    # 출력 임베딩 크기 지정 : 0=기본 모델 임베딩크기(768), 예:128=128 츨력임베딩 크기 

word_embedding_model, bi_encoder = bi_encoder(model_path=bi_encoder_path, max_seq_len=512, do_lower_case=True, pooling_mode=pooling_mode, out_dimension=out_dimension, device=device)
#bi_encoder.to(device)

# cross-encoder 모델 로딩
if use_cross_encoder == True:
    cross_encoder_path = "bongsoo/klue-cross-encoder-v1"# klue-cross-encoder-v1 # albert-small-kor-cross-encoder-v1 # kpf-cross-encoder-v1
    #cross_encoder_path = "../../../data11/model/moco/cross/kpf-cross-sts4"
    cross_encoder = CrossEncoder(cross_encoder_path, max_length=512, num_labels=1, device=device)

In [ ]:
#-------------------------------------------------------------------------------------------------------
# 2. KorQuAD_v1.0_dev.json 파일 로딩 하여, 각 항목별 리스트를 출력한후, contexts df, questions df를 만듬.
#-------------------------------------------------------------------------------------------------------
from myutils import read_korquad_v1_json, read_aihub_qua_json

# aihub QuA 파일을 불러옴.
jsonfile = './data/VL_unanswerable.json' # VL_unanswerable.json # VL_text_entailment.json # VL_span_inference.json # KorQuAD_v1.0_dev.json
contexts, questions, answers, contextids, qcontextids = read_aihub_qua_json(jsonfile) # read_aihub_qua_json(jsonfile)

# list들을 zip 으로 묶고, dataframe 생성함
# context, contextid를 묶어서 context df 만듬.
df_contexts = pd.DataFrame((zip(contexts, contextids)), columns = ['context','contextid'])

# question, answer, contextids를 묶어서 question df 만듬
df_questions = pd.DataFrame((zip(questions, answers, qcontextids)), columns = ['question','answer', 'contextid'])

In [ ]:
df_contexts.head()

In [ ]:
df_contexts[0:1].context.values[0]

In [ ]:
df_questions.head()

In [ ]:
#------------------------------------------------------
# faiss 인덱스 생성 테스트
#------------------------------------------------------
from myutils import embed_text_avg, embed_vocab_bytag
from sklearn.metrics.pairwise import cosine_similarity

paragraphs = df_contexts.context.to_list()
#print(paragraphs)

embeddings = [embed_text_avg(model=bi_encoder, paragraph=paragraph) for paragraph in tqdm(paragraphs)]

embeddings_arr = np.array([embedding for embedding in embeddings]).astype("float32")#float32 로 embeddings 타입 변경
    
index_test = faiss.IndexFlatIP(embeddings_arr.shape[1])
faiss.normalize_L2(embeddings_arr)# *cosine유사도 구할때는 반드시 normalize 처리함.
    
# id를 매핑 시켜줌 => 이때 idtype은 반드시 int64 type이어야 함.
index_test = faiss.IndexIDMap2(index_test)

index_test.add_with_ids(embeddings_arr, df_contexts.contextid.values)

In [ ]:
#------------------------------------------------------
# faiss 인덱스 서치 테스트
#------------------------------------------------------

from myutils import embed_text, embed_vocab_bytag

query = '서울지방결창청 공안부실.'  #쿼리
q1 = bi_encoder.encode(query)
#q1 = embed_vocab_bytag(model=bi_encoder, paragraph=[query], show=True)

vector = np.array([q1]).astype('float32')
#print(vector.shape)
faiss.normalize_L2(vector)   

distance, idx = index_test.search(vector, k=30)
print(f'{distance}-{idx}')

In [ ]:
#------------------------------------------------------
# sklearn 코사인 테스트
#------------------------------------------------------
from myutils import embed_text_avg, embed_vocab_bytag
from sklearn.metrics.pairwise import cosine_similarity

cid = 372  # 검색된 contexts idx-10000 해준 값 입력.

#embed1 = embed_vocab_bytag(model=bi_encoder, paragraph=[df_contexts[cid-1:cid].context.values[0]], show=True) # 단어 평균 임베딩
embed1 = embed_text_avg(model=bi_encoder, paragraph=df_contexts[cid-1:cid].context.values[0])                  # 문장 평균 임베딩

#embed2 = embed_vocab_bytag(model=bi_encoder, paragraph=[query], show=True)       # 단어 평균 쿼리
embed2 = bi_encoder.encode(query)                                                 # 문장평균 쿼리 

cosine_sim = cosine_similarity([embed1], [embed2]) # (1,768) 식에 2차원 배열입력되어야 함.
print(f'*유사도:{cosine_sim}')
print(df_contexts[cid-1:cid].context.values)

In [ ]:
#---------------------------------------------------------
# 3. 임베딩 벡터 생성 후 FAISS에 인덱싱함.
#---------------------------------------------------------
from myutils import embed_text_avg, embed_vocab_bytag
   
#=============================================================================
# 임베딩 벡터 생성하여 FAISS에 인덱싱하고 ID와 매핑 처리하는 함수
# => IN : contextdf
# => OUT : FASSI index
#=============================================================================
def embeddingforfaiss(df, embed_avg_method:int ):
           
    # embedding 생성(인코딩)
    start = time.time()
    paragraphs = df.context.to_list()
    print(f'*임베딩 할 context 계수: {len(paragraphs)}') 
    
    # paragraph_avg == True 이면 평균 문자 임베딩 벡터 구함.
    if embed_avg_method == 1:
        print(f'*----문장들 평균 임베딩 벡터 구하기----')
        embeddings = [embed_text_avg(model=bi_encoder, paragraph=paragraph) for paragraph in tqdm(paragraphs)]
    elif embed_avg_method == 2:
        print(f'*----단어들 평균 임베딩 벡터 구하기----')
        embeddings = [embed_vocab_bytag(model=bi_encoder, paragraph=paragraph) for paragraph in tqdm(paragraphs)]
    else:
        print(f'*----문단 벡터 구하기----')
        embeddings = bi_encoder.encode(paragraphs, show_progress_bar=True, convert_to_tensor=False)
       
    embeddings_arr = np.array([embedding for embedding in embeddings]).astype("float32")#float32 로 embeddings 타입 변경
    #print(type(embeddings)) #print(embeddings.shape) #print(embeddings[0])    
    
    # instance index 생성
    # => IndexFlatL2 : Euclidean Distance 측정함
    # => IndexFlatIP : cosine 유사도 측정함 => faiss.normalize_L2(embeddings) 호출해줘야 함.
    if faiss_index_method == 0:      # 0=Cosine Similarity 적용(IndexFlatIP 사용), 
        index = faiss.IndexFlatIP(embeddings_arr.shape[1])
        faiss.normalize_L2(embeddings_arr)# *cosine유사도 구할때는 반드시 normalize 처리함.
    elif faiss_index_method == 1:    # 1=Euclidean Distance 적용(IndexFlatL2 사용)
        index = faiss.IndexFlatL2(embeddings_arr.shape[1])
    
    # id를 매핑 시켜줌 => 이때 idtype은 반드시 int64 type이어야 함.
    index = faiss.IndexIDMap2(index)

    index.add_with_ids(embeddings_arr, df.contextid.values)

    print(f'*임베딩 시간 : {time.time()-start:.4f}')
    
    return index

#df 임베딩 구하고 faiss에 추가함.
index = embeddingforfaiss(df_contexts, embed_avg_method = embed_avg_method)

In [ ]:
#----------------------------------------------------------------
# 4. 쿼리를 list로 만들고, 검색 후 예측된 결과를 list로 만듬
#----------------------------------------------------------------
from tqdm.notebook import tqdm
from myutils import df_sampling, embed_vocab_bytag

# Query를 list로 만들고 -> query 인코딩후->검색 결과 출력

if query_num == 0:   # query_num = 0 이면 모든 쿼리(5533개)
    user_query = df_questions['question'].values.tolist()
else:   # query_num > 0이면 해당 계수만큼 랜덤하게 샘플링하여 쿼리 목록을 만듬.
    df_questions = df_sampling(df=df_questions, num=query_num, seed=seed)
    user_query = df_questions['answer'].values.tolist()

print(f'Query-----------------------------------------------------')
print(user_query[0:3])

# 단어 평균인 경우에는 쿼리도 평균으로 구함.
if embed_avg_method == 2:   
    print(f'*----단어들 평균 임베딩 벡터 구하기----')
    vector_list = []
    for uquery in user_query:
        tmp = embed_vocab_bytag(model=bi_encoder, paragraph=uquery)
        vector_list.append(tmp) 
        
    vector = np.array(vector_list).astype('float32')
else:
    vector = bi_encoder.encode(user_query)
    
if faiss_index_method == 0:
    faiss.normalize_L2(vector)              # *cosine유사도 구할때는 반드시 normalize 처리함.
    
distance, idx = index.search(np.array(vector).astype("float32"), k=search_k)

# 예측검색결과를 리스트로 만듬.
bi_predictions_list = []

for i, query in enumerate(tqdm(user_query)):
    bi_predictions_list.append(idx[i].tolist())
    
print(f'----------------------------------------------------------')
print(f'bi-encoder 예측:{len(bi_predictions_list)}')
print(bi_predictions_list[0:3])

# cross-encoder 사용인 경우에
# - 한번더 검색된 결과에 {쿼리, 문장} 쌍으로 만들어서 cross-encoder로 스코어 출력함.
if use_cross_encoder == True:
    # {쿼리, 문장} 쌍 만듬.
    #count = 0
    cross_predictions_list = []
    for i, predicts in enumerate(tqdm(bi_predictions_list)):
        sentence_combinations = []
        query = user_query[i]
        #count += 1
             
        for predict in predicts:  
             # {쿼리, 문장} 쌍을 만듬 (예: ['프랑스 해안에서 발견된 고래는 뭐야?, '프랑스 남부 니츠 해안에서는 지난 2022년 10월 1일 커다른 물고기 시체가 떠왔다.....']
             sentence_combinations.append([query, df_contexts[df_contexts.contextid == predict]['context'].values.tolist()[0]])
                     
        # cross-enocoder 돌려서 출력된 score 추가함.
        cross_scores = cross_encoder.predict(sentence_combinations)+1
        #print(f'*cross_scores:{cross_scores}')
        
        dec_cross_idx = reversed(np.argsort(cross_scores))
        tmp_list = []
        for idx in dec_cross_idx:
            #print(f'idx:{idx}-predicts:{predicts[idx]}')
            tmp_list.append(predicts[idx])
         
        cross_predictions_list.append(tmp_list)   
    
    print(f'----------------------------------------------------------')
    print(f'cross-encoder 예측:{len(cross_predictions_list)}')
    print(f'{cross_predictions_list[0:3]}')


In [ ]:
#--------------------------------------------------------------------------------------------------
# 5. MRR을 구함
##--------------------------------------------------------------------------------------------------
from myutils import mean_reciprocal_rank

# 정답, 여기서는 contextid를 리스트로 만듬.
ground_truths_list = df_questions['contextid'].values.tolist()
#print(f'gtlen:{len(ground_truths_list)}')
#print(ground_truths_list[0:9])

logger.info(f'--------------------------------------------------------------------------')
logger.info('json_file:{}'.format(jsonfile))
logger.info(f'*faiss 인덱싱 방식: {faiss_index_method}(0=코사인, 1=유클리드)')
logger.info(f'*문단 임베딩 방식: {embed_avg_method}(0=전체, 1=문장평균, 2=단어평균)')
logger.info('*search_k:{}/query_num:{}'.format(search_k, query_num))

# MRR를 구함
if use_cross_encoder == True:
    predictions_list = bi_predictions_list
    bi_ranks, bi_score = mean_reciprocal_rank(ground_truths_list, predictions_list)

    # BI-MRR 출력
    logger.info(f'--------------------------------------------------------------------------')
    logger.info('*BI-ENCODER:{}'.format(bi_encoder_path))
    logger.info('*BI-MRR:{:.4f}'.format(bi_score))
    logger.info(f'*Ranks({len(bi_ranks)}):{bi_ranks[0:20]}')
    
    predictions_list = cross_predictions_list
    cross_ranks, cross_score = mean_reciprocal_rank(ground_truths_list, predictions_list)
    logger.info(f'---------------------------------------------------------------------------')
    logger.info('*CROSS-ENCODER:{}'.format(cross_encoder_path))
    logger.info('*CROSS-MRR:{:.4f}'.format(cross_score))
    logger.info(f'*Ranks({len(cross_ranks)}):{cross_ranks[0:20]}')
    
    # 검색 한 계슈
    #logger.info(f'---------------------------------------------------------------------------')
    search_count = 0
    for item in cross_ranks:
        if item != 0:
            search_count += 1

    logger.info('*검색률: {}/{}({:.2f}%)'.format(search_count, len(cross_ranks), (search_count/len(cross_ranks))*100))
    logger.info(f'---------------------------------------------------------------------------')

    
    logger.info(f'\nBI -> CROSS---------------------------------------------------------------------------')
    df_scores = pd.DataFrame((zip(bi_ranks, cross_ranks)), columns = ['bi-rank','cross-rank'])
    logger.info(df_scores.head(20))

else:
    predictions_list = bi_predictions_list

    bi_ranks, bi_score = mean_reciprocal_rank(ground_truths_list, predictions_list)

    # BI-MRR 출력
    logger.info(f'----------------------------------------------------------------------------')
    logger.info('*BI-ENCODER:{}'.format(bi_encoder_path))
    logger.info('*BI-MRR:{:.4f}'.format(bi_score))
    logger.info(f'*Ranks({len(bi_ranks)}):{bi_ranks[0:20]}')
    
    # 검색 한 계슈
    #logger.info(f'---------------------------------------------------------------------------')
    search_count = 0
    for item in bi_ranks:
        if item != 0:
            search_count += 1

    logger.info('*검색률: {}/{}({:.2f}%)'.format(search_count, len(bi_ranks), (search_count/len(bi_ranks))*100))
    logger.info(f'---------------------------------------------------------------------------')


In [ ]:
#------------------------------------------------------------------------------------------------
# 6. BM25 계산
# => korquad_v1.0 말뭉치를 가지고 BM25 계산하는 함수
# => for문을 2번 돌면서 BM25 2번 계산함.
#    - 처음에는(0) 해당 쿼리와 contexts에 대해 mecab 적용 후 BM25 스코어 계산, 
#    - 2번째는 mecab 적용하지 않고 계산
#------------------------------------------------------------------------------------------------
if use_bm25 == True:

    import konlpy
    from konlpy.tag import Mecab
    from rank_bm25 import BM25Okapi
    from tqdm.notebook import tqdm

    def BM25tokenizer(sent):
      return sent.split(" ")

    # 입력된 contexts를 mecab을 이용하여 형태소 추출 후 " " 붙여서 형태소 문장을 만듬.
    # Mecab 선언
    mecab = Mecab()

    for count in range(2):
        mecab_str = ''
        # 0이면(처음엔) mecab 적용함=> tokeniaer 후 인덱싱
        if count == 0:
            mecab_str = "(mecab 적용)"
            mecab_contexts=[]
            for context in tqdm(contexts):
                temp = mecab.morphs(context)   # ['세계', '배달', '피자', '리더', '도미노피자','가'..] 식으로 temp 리스트가 생성됨
                sentence = " ".join(temp)      # 위 temp 리스트를 공백을 넣어서 한문장으로 합침 ['세계 배달 피자 리더 도미노피자 가 ...]
                mecab_contexts.append(sentence)

            print(f'*contexts_len:{len(mecab_contexts)}')  
            print(f'{mecab_contexts[0]}')

            # tokeniaer 후 인덱싱
            tokenized_corpus = [BM25tokenizer(doc) for doc in mecab_contexts]
        else:
            # 2번째는 그냥 인덱싱
            tokenized_corpus = [BM25tokenizer(doc) for doc in contexts]
            
        bm25 = BM25Okapi(tokenized_corpus)

        #print(f'bm25.doc_len:{bm25.doc_len}')
        #print(f'type(bm25.doc_freqs):{type(bm25.doc_freqs)}')
        
        bm5_predictions_list = []
     
        # 쿼리 후 get_scores 를 이용하여, scores를 구함.
        for idx, query in enumerate(user_query):
            
            # 처음에는 mecab적용해서 query문 전처리 함.
            if count == 0:
                tempq = mecab.morphs(query) 
                query = " ".join(tempq)
                
            # 쿼리에 따른 스코어 구함    
            tokenized_query = BM25tokenizer(query)
            doc_scores = bm25.get_scores(tokenized_query)

            # 정렬후 최대 스코어 search_k 만큼만 출력함
            top_lists = sorted(enumerate(doc_scores), key=lambda x: x[1], reverse=True)[:search_k]
            bm5_predictions_list.append([index + contextids[0] for index, score in top_lists])

        # 정답, 여기서는 contextid를 리스트로 만듬.
        ground_truths_list = df_questions['contextid'].values.tolist()

        # MPR 계산
        predictions_list = bm5_predictions_list  # 예측 결과 리스트
        bm25_ranks, bm25_score = mean_reciprocal_rank(ground_truths_list, predictions_list)

         # BM25-MRR 출력
        logger.info(f'--------------------------------------------------------------------------')
        logger.info('*BM25-MRR{}:{:.4f}'.format(mecab_str, bm25_score))
        logger.info(f'*Ranks({len(bm25_ranks)}):{bm25_ranks[0:20]}')
        # 검색 한 계슈
        #logger.info(f'---------------------------------------------------------------------------')
        search_count = 0
        for item in bm25_ranks:
            if item != 0:
                search_count += 1

        logger.info('*BM25{} 검색률: {}/{}({:.2f}%)'.format(mecab_str, search_count, len(bm25_ranks), (search_count/len(bm25_ranks))*100))
        logger.info(f'---------------------------------------------------------------------------')